In [2]:
%load_ext autoreload
%autoreload 2

import logging
logging.getLogger("querychains").setLevel(logging.DEBUG)
import querychains
from querychains import with_context, Context, FileStorage, start_server

In [4]:
storage = FileStorage("data")
storage

<FileStorage directory='/home/spirali/projects/querychain/data'>

In [3]:
# Register context into storage

with Context("root1", storage=storage):
    pass


In [4]:
# Manual writing context into storage

with Context("root2") as c:
    pass

storage.write_context(c)

In [6]:
# Composing directory structure with contexts

with Context("root3", storage=storage, directory=True):
    with Context("first child", directory=True):
        with Context("a"):
            pass
        with Context("b"):
            pass
    with Context("second child", directory=True):
        with Context("a"):
            pass
        with Context("b"):
            pass

import pathlib
list(pathlib.Path(f"data/{}").rglob("*"))

[PosixPath('data/389b6db6-7e2e-4fee-9ef8-69551045a326.ctx'),
 PosixPath('data/659cd046-22c2-4d7d-8c30-08141a65b1ee.ctx'),
 PosixPath('data/389b6db6-7e2e-4fee-9ef8-69551045a326.ctx/49f557d6-4708-4648-8655-f53a10f1125b.ctx'),
 PosixPath('data/389b6db6-7e2e-4fee-9ef8-69551045a326.ctx/_self.json.gz'),
 PosixPath('data/389b6db6-7e2e-4fee-9ef8-69551045a326.ctx/7a5e7b3b-a291-4b5c-9065-6739ab76c51a.ctx'),
 PosixPath('data/389b6db6-7e2e-4fee-9ef8-69551045a326.ctx/49f557d6-4708-4648-8655-f53a10f1125b.ctx/6ec462b6-d29d-4c42-90cf-c4bdb5328911.json.gz'),
 PosixPath('data/389b6db6-7e2e-4fee-9ef8-69551045a326.ctx/49f557d6-4708-4648-8655-f53a10f1125b.ctx/_self.json.gz'),
 PosixPath('data/389b6db6-7e2e-4fee-9ef8-69551045a326.ctx/49f557d6-4708-4648-8655-f53a10f1125b.ctx/8800260b-f5f4-4bbc-ae0f-8db2fc120d21.json.gz'),
 PosixPath('data/389b6db6-7e2e-4fee-9ef8-69551045a326.ctx/7a5e7b3b-a291-4b5c-9065-6739ab76c51a.ctx/e04eb1b1-b83b-4a51-a2fe-710af00a1336.json.gz'),
 PosixPath('data/389b6db6-7e2e-4fee-9ef8-6

In [5]:
# Running data browser over storage

server = start_server(storage=storage)

INFO:     Started server process [26052]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:35627 (Press CTRL+C to quit)
